In [1]:
# import libraries
import json
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

/home/nrabbi/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# read the data
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
# get the raw data on the same level
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
# initialize the model
model = SentenceTransformer("all-mpnet-base-v2")

In [5]:
# find the dimension of the model
len(model.encode("This is a simple sentence"))

768

In [6]:
# create embeddings using the model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [7]:
es_client = Elasticsearch('http://localhost:9200') 
es_client.info()

ObjectApiResponse({'name': 'f4b2dfd7c40c', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'JNciRQjyR8qhFx2bV4VyIA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [8]:
# provide the mapping for the index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [9]:
# initialize the index
index_name = "course-questions"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [10]:
# index the documents
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [11]:
# create embeddings for the search query
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [12]:
# building the query
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5, # number of documents to return
    "num_candidates": 10000, 
}

In [13]:
# perform the search
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'NAobn5ABsEw5DSO7gOz2',
  '_score': 0.7147918,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'Rwobn5ABsEw5DSO7ou_c',
  '_score': 0.6134736,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares at

In [14]:
# build a new query for a filtered search
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [15]:
# perform the filtered search
response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"}
    },
    knn=knn_query,
    size=5
)

# see the results
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'NAobn5ABsEw5DSO7gOz2',
  '_score': 11.603098,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.02696547843515873,
    -0.0006259932997636497,
    -0.016629500314593315,
    0.052851494401693344,
    0.05476532131433487,
    -0.03133988752961159,
    0.029942717403173447,
    -0.04808562621474266,
    0.04467545449733734,
    0.005839466117322445,
    0.016233088448643684,
    0.012001161463558674,
    -0.03122236765921116,
    0.0166005901992321,
    -0.04886902868747711,
    -0.06496307998895645,
    0.0464341901242733,
    -0.009297674521803856,
    -0.06425288319587708,
    -0.013732724823057652,
    -0.01597619242966175,
    0.008629505522549152,
    -0.0244789514690637